# Installation

Installation performed on Python 3.8.19.

## Prerequisities

When performed on WLSL:
* Install WSLS Cuda: https://www.youtube.com/watch?v=JaHVsZa2jTc

Install libs (versions may change especially if MMOCR upgrades from 1.0.1 - it was tested on 1.0.1):

In [2]:
%pip install "torch==2.0.0+cu118" "torchvision==0.15.1+cu118" -f https://download.pytorch.org/whl/torch_stable.html
%pip install -U openmim
!mim install "mmengine>=0.7.1,<1.1.0"
!mim install "mmcv>=2.0.0rc4,<2.1.0"
!mim install "mmdet>=3.0.0rc5,<3.2.0"

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 55.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1
    Uninstalling torchvision-0.15.1:
      Successfully uninstalled torchvision-0.15.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html


Install nvidia-cuda-toolkit: `apt install nvidia-cuda-toolkit`

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


## MMOCR

### Clone and install from github

Do not download it as lib, MMOCR states that it should be compiled from source!

In [7]:
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
%pip install -v -e .
# "-v" increases pip's verbosity.
# "-e" means installing the project in editable mode,
# That is, any local modifications on the code will take effect immediately.

Cloning into 'mmocr'...
remote: Enumerating objects: 16384, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 16384 (delta 101), reused 200 (delta 70), pack-reused 16091
Receiving objects: 100% (16384/16384), 16.24 MiB | 21.76 MiB/s, done.
Resolving deltas: 100% (10769/10769), done.
/home/kaybenot/MMOCR/mmocr
Using pip 23.3.1 from /home/kaybenot/MMOCR/.conda/lib/python3.8/site-packages/pip (python 3.8)
Obtaining file:///home/kaybenot/MMOCR/mmocr
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-dynf1grr/mmocr.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-dynf1grr/mmocr.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-dynf1grr/mmocr.egg-info/SOURCES.txt'
done
  Running setup.py develop for mmocr
    Running command python setup.py develop
    running develop
    /home/kaybenot/MMOCR/.conda/lib/python3.8/site-p

### Test if installed correcty

General test:

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check mmocr installation
import mmocr
print(mmocr.__version__)

%cd ./mmocr/
!ls

2.0.0+cu118 True
3.1.0
2.0.1
11.7
GCC 9.3
1.0.1
/home/kaybenot/MMOCR/mmocr
CITATION.cff  README_zh-CN.md  dicts   mmocr.egg-info	requirements.txt  tests
LICENSE       configs	       docker  model-index.yml	resources	  tools
MANIFEST.in   dataset_zoo      docs    projects		setup.cfg
README.md     demo	       mmocr   requirements	setup.py


Cuda test:

In [3]:
import torch

print(f"{torch.cuda.is_available()=}")

f = torch.nn.Conv2d(3, 8, 3, device="cuda")
X = torch.randn(2, 3, 4, 4, device="cuda")

Y = X @ X
print(f"{Y.shape=}")
print("matrix multiply works")

Y = f(X)
print(f"{Y.shape=}")
print("Conv2d works")

torch.cuda.is_available()=True
Y.shape=torch.Size([2, 3, 4, 4])
matrix multiply works
Y.shape=torch.Size([2, 8, 2, 2])
Conv2d works


MMOCR demo:

In [3]:
from mmocr.apis import MMOCRInferencer
ocr = MMOCRInferencer(det='DBNet', rec='CRNN')
ocr('demo/demo_text_ocr.jpg', show=True, print_result=True)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textdet/dbnet/dbnet_resnet50-oclip_1200e_icdar2015/dbnet_resnet50-oclip_1200e_icdar2015_20221102_115917-bde8c87a.pth
04/25 14:27:36 - mmengine - WARNING - Failed to search registry with scope "mmocr" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmocr" is a correct scope, or whether the registry is initialized.
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/crnn/crnn_mini-vgg_5e_mj/crnn_mini-vgg_5e_mj_20220826_224120-8afbedbb.pth


/home/kaybenot/MMOCR/.conda/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



Output()

{'predictions': [{'rec_texts': ['cbanks', 'docecea', 'grouf', 'pwate', 'chobnsonsg', 'soxee', 'oeioh', 'c', 
'sones', 'lbrandec', 'sretalg', '11', 'to8', 'round', 'sale', 'year', 'ally', 'sie', 'sall'], 'rec_scores': 
[0.867136667172114, 0.7639567128249577, 0.5317003488540649, 0.22387954890727996, 0.7914273649454117, 
0.376357501745224, 0.3872466623783112, 0.1318681836128235, 0.6457703858613968, 0.6825660988688469, 
0.709580796105521, 0.6951165497303009, 0.5226845443248749, 0.9999941825866699, 0.916475921869278, 
0.9982690066099167, 0.9997920244932175, 0.6453402439753214, 0.9325271546840668], 'det_polygons': 
[[551.9930285844646, 411.9138765335083, 553.6153911653112, 383.53195309638977, 620.2410061195247, 
387.33785033226013, 618.6186435386782, 415.71977376937866], [468.9537496097752, 406.6847491264343, 
474.2110249003425, 373.215206861496, 568.4817622450531, 388.01310658454895, 563.2244869544857, 421.4826488494873], 
[590.5385245901639, 330.01953125, 590.5385245901639, 312.1484375, 637.6147540983605, 312.1484375, 
637.6147540983605, 330.01953125], [631.0712565437691, 314.3289130926132, 681.7556297427317, 310.60336887836456, 
682.9117894907466, 326.32201969623566, 632.2274162917839, 330.0475639104843], [498.0933179511398, 321.518217921257,
499.76674534531884, 304.2317560315132, 588.7035374375639, 312.8356558084488, 587.0301100433849, 
330.12213587760925], [439.82545626280734, 310.3056991100311, 442.90629392530093, 292.7744150161743, 
498.7986234070824, 302.5900638103485, 495.7177857445888, 320.1213479042053], [637.2627560474832, 290.9204235672951,
675.6445827796811, 289.9612048268318, 676.1046025510693, 308.3569383621216, 637.7227758188716, 309.31615710258484],
[619.7377049180327, 309.169921875, 619.7377049180327, 290.703125, 639.4024590163933, 290.703125, 639.4024590163933,
309.169921875], [580.587578245069, 308.2032859325409, 581.5090726258323, 287.93710857629776, 623.0820864818135, 
289.82618629932404, 622.1605921010502, 310.0923454761505], [516.6630518678759, 300.6750440597534, 
518.5960223588786, 282.3175781965256, 581.720787923844, 288.96008253097534, 579.7878174328413, 307.3175483942032], 
[462.0407595775166, 295.54255843162537, 463.860179963659, 277.6142358779907, 517.9023919777791, 283.0949664115906, 
516.0829715916367, 301.0232889652252], [445.13852459016385, 288.916015625, 445.13852459016385, 275.21484375, 
462.4196721311475, 275.21484375, 462.4196721311475, 288.916015625], [12.708091648289413, 242.55853950977325, 
14.11987552330142, 208.68708670139313, 74.6477825602547, 211.20824456214905, 73.23599868524269, 
245.07969737052917], [450.07097092925517, 223.7011033296585, 453.98244834024393, 179.38590288162231, 
660.8845967277151, 197.63589203357697, 656.9731193167263, 241.95109248161316], [18.766596228177427, 
114.42952916026115, 20.849587793819236, 88.05377706885338, 83.81092504282465, 93.02275702357292, 81.72792438444543,
119.3985091149807], [588.0643543681159, 116.80935114622116, 592.6805552998526, 67.46757835149765, 
707.6868075011206, 78.21997612714767, 703.0706065693838, 127.56174892187119], [457.36167327380565, 
107.18433171510696, 461.6064540675428, 54.958303570747375, 588.3892560614913, 65.25597959756851, 584.1444752677542,
117.4820077419281], [31.773472954796958, 58.521878272295, 32.824093299615576, 39.61701199412346, 68.73221126618931,
41.6112394630909, 67.6815909213707, 60.516105741262436], [156.79603691726433, 39.05041866004467, 
159.12410503449985, 11.898614540696144, 217.97542949739048, 16.941330209374428, 215.64736138015493, 
44.093134328722954]], 'det_scores': [0.8227497935295105, 0.7761126756668091, 0.770125687122345, 0.7366573214530945,
0.8219332098960876, 0.7992833256721497, 0.8390302658081055, 0.7473655343055725, 0.8208408355712891, 
0.8446050882339478, 0.8288700580596924, 0.6628001928329468, 0.7784616947174072, 0.9304829835891724, 
0.8886011242866516, 0.9348414540290833, 0.8919371366500854, 0.8632858991622925, 0.9080162644386292]}]}

{'predictions': [{'rec_texts': ['cbanks',
    'docecea',
    'grouf',
    'pwate',
    'chobnsonsg',
    'soxee',
    'oeioh',
    'c',
    'sones',
    'lbrandec',
    'sretalg',
    '11',
    'to8',
    'round',
    'sale',
    'year',
    'ally',
    'sie',
    'sall'],
   'rec_scores': [0.867136667172114,
    0.7639567128249577,
    0.5317003488540649,
    0.22387954890727996,
    0.7914273649454117,
    0.376357501745224,
    0.3872466623783112,
    0.1318681836128235,
    0.6457703858613968,
    0.6825660988688469,
    0.709580796105521,
    0.6951165497303009,
    0.5226845443248749,
    0.9999941825866699,
    0.916475921869278,
    0.9982690066099167,
    0.9997920244932175,
    0.6453402439753214,
    0.9325271546840668],
   'det_polygons': [[551.9930285844646,
     411.9138765335083,
     553.6153911653112,
     383.53195309638977,
     620.2410061195247,
     387.33785033226013,
     618.6186435386782,
     415.71977376937866],
    [468.9537496097752,
     406.6847491264343